In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from scipy.spatial.distance import cosine
import os

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define ResNet Architecture
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.conv1 = conv3x3(3, 64)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0])
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride=1):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        out = F.relu(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, start_dim=1)
        out = self.linear(out)
        return out

def resnet18(num_classes=100):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)

# Load Client 14's Data
data_path = f"client_14.pt"
if not os.path.exists(data_path):
    raise FileNotFoundError(f"Client 14's dataset {data_path} not found!")

client_data = torch.load(data_path)
client_train_data, client_train_targets = client_data['data'], client_data['targets']

# DataLoader for Client 14
batch_size = 128
client_train_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(client_train_data, client_train_targets),
    batch_size=batch_size, shuffle=True
)


model_filenames = [
# based on the precious stage, put round 3's Top-K model here 
]


models = []
for filename in model_filenames:
    model = resnet18(num_classes=100).to(device)
    checkpoint = torch.load(f'./checkpoint/{filename}')
    model.load_state_dict(checkpoint['net'])
    model.eval()
    models.append(model)

# Define the Meta-Model (Weighted Ensemble)
class MetaModel(nn.Module):
    def __init__(self, num_models):
        super(MetaModel, self).__init__()
        self.weights = nn.Parameter(torch.ones(num_models) / num_models)

    def forward(self, outputs):
        weighted_outputs = torch.stack([w * o for w, o in zip(self.weights, outputs)], dim=0)
        return torch.sum(weighted_outputs, dim=0)

# Train Meta-Model (Optimize Ensemble Weights)
def train_meta_model(meta_model, models, train_loader, criterion, optimizer, num_epochs=200, patience=20):
    best_acc = 0.0
    patience_counter = 0
    for epoch in range(num_epochs):
        meta_model.train()
        total_loss, correct, total = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = [model(inputs) for model in models]
            optimizer.zero_grad()
            ensemble_output = meta_model(outputs)
            loss = criterion(ensemble_output, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(ensemble_output, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        avg_train_loss = total_loss / total
        train_acc = correct / total
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Acc: {train_acc:.2f}%')

        if train_acc > best_acc:
            torch.save(meta_model.state_dict(), './weight_checkpointsdir/meta_weights_client14.pth')
            best_acc = train_acc
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print('Early stopping triggered')
            break

meta_model = MetaModel(num_models=len(models)).to(device)
optimizer = optim.SGD(meta_model.parameters(), lr=0.01, momentum=0.9)
criterion = nn.CrossEntropyLoss()
train_meta_model(meta_model, models, client_train_loader, criterion, optimizer)

print("Meta-Model Training Complete!")
